In [1]:
from pyscf import gto, scf, mp, cc
# atoms = '''
#     O   -1.485163346097   -0.114724564047    0.000000000000
#     H   -1.868415346097    0.762298435953    0.000000000000
#     H   -0.533833346097    0.040507435953    0.000000000000
#     O    1.416468653903    0.111264435953    0.000000000000
#     H    1.746241653903   -0.373945564047   -0.758561000000
#     H    1.746241653903   -0.373945564047    0.758561000000
# '''
basis = 'sto6g'
a = 1.
nH = 9
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis=basis, spin=1, verbose=4, max_memory=16000)
mf = scf.UHF(mol).density_fit()
mf.kernel()

nfrozen = 0
# canonical
# mmp = mp.UMP2(mf, frozen=nfrozen)
# mmp.kernel()

# mycc = cc.UCCSD(mf, frozen=nfrozen)
# eris = mycc.ao2mo()
# mycc.kernel(eris=eris)
# et = mycc.ccsd_t(eris=eris)

System: uname_result(system='Linux', node='sharmagroup-rn', release='6.14.0-36-generic', version='#36~24.04.1-Ubuntu SMP PREEMPT_DYNAMIC Wed Oct 15 15:45:17 UTC 2', machine='x86_64')  Threads 16
Python 3.11.14 (main, Oct 21 2025, 18:31:21) [GCC 11.2.0]
numpy 2.3.1  scipy 1.16.2  h5py 3.14.0
Date: Fri Dec  5 14:15:25 2025
PySCF version 2.11.0
PySCF path  /home/sharmagroup/sharmagroup/pyscf
GIT HEAD (branch master) 3d1768f5e33b144b606c3d2c81c12ee54d794501

[ENV] PYSCF_EXT_PATH /home/sharmagroup/sharmagroup/pyscf-forge
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 9
[INPUT] num. electrons = 9
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 1
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.000000000000

In [2]:
from pyscf import lo
import numpy as np
orbocca = mf.mo_coeff[0][:,nfrozen:np.count_nonzero(mf.mo_occ[0])]
orbloca = lo.PipekMezey(mol, orbocca).kernel()
orboccb = mf.mo_coeff[1][:,nfrozen:np.count_nonzero(mf.mo_occ[1])]
orblocb = lo.PipekMezey(mol, orboccb).kernel()
lo_coeff = [orbloca, orblocb]

oa = [[[i],[]] for i in range(orbloca.shape[1])]
ob = [[[],[i]] for i in range(orblocb.shape[1])]
frag_lolist = oa + ob

for ifrag,loidx in enumerate(frag_lolist):
    print(ifrag,loidx)
    orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]



******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
pop_method = meta_lowdin
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 2.6983385194866  delta_f= 2.69834  |g|= 0.151192  1 KF 3 Hx
macro= 2  f(x)= 2.6983386149813  delta_f= 9.54947e-08  |g|= 0.000825343  1 KF 2 Hx
macro= 3  f(x)= 2.6983386149813  delta_f= 4.44089e-16  |g|= 1.89299e-05  1 KF 1 Hx
macro X = 3  f(x)= 2.6983386149813  |g|= 1.89299e-05  6 intor 3 KF 6 Hx


******** <class 'pyscf.lo.pipek.PipekMezey'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.05
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14


In [3]:
from pyscf import lib
from pyscf.lno import ulnoccsd
def ulno_ccsd(mcc, mo_coeff, uocc_loc, mo_occ, maskact, ccsd_t=False):

    occidxa = mo_occ[0]>1e-10
    occidxb = mo_occ[1]>1e-10
    nmo = mo_occ[0].size, mo_occ[1].size
    moidxa, moidxb = maskact

    orbfrzocca = mo_coeff[0][:, ~moidxa &  occidxa]
    orbactocca = mo_coeff[0][:,  moidxa &  occidxa]
    orbactvira = mo_coeff[0][:,  moidxa & ~occidxa]
    orbfrzvira = mo_coeff[0][:, ~moidxa & ~occidxa]
    nfrzocca, nactocca, nactvira, nfrzvira = [orb.shape[1]
                                              for orb in [orbfrzocca,orbactocca,
                                                          orbactvira,orbfrzvira]]
    orbfrzoccb = mo_coeff[1][:, ~moidxb &  occidxb]
    orbactoccb = mo_coeff[1][:,  moidxb &  occidxb]
    orbactvirb = mo_coeff[1][:,  moidxb & ~occidxb]
    orbfrzvirb = mo_coeff[1][:, ~moidxb & ~occidxb]
    nfrzoccb, nactoccb, nactvirb, nfrzvirb = [orb.shape[1]
                                              for orb in [orbfrzoccb,orbactoccb,
                                                          orbactvirb,orbfrzvirb]]
    nlo = [uocc_loc[0].shape[1], uocc_loc[1].shape[1]]
    prjlo = [uocc_loc[0].T.conj(), uocc_loc[1].T.conj()]

    if nactocca * nactvira == 0 and nactoccb * nactvirb == 0:
        elcorr_pt2 = lib.tag_array(0., spin_comp=np.array((0., 0.)))
        elcorr_cc = lib.tag_array(0., spin_comp=np.array((0., 0.)))
        elcorr_cc_t = 0.
    else:
        # solve impurity problem
        imp_eris = mcc.ao2mo()
        # MP2 fragment energy
        t1, t2 = mcc.init_amps(eris=imp_eris)[1:]
        elcorr_pt2 = ulnoccsd.get_fragment_energy(imp_eris, t1, t2, prjlo)
        # CCSD fragment energy
        t1, t2 = mcc.kernel(eris=imp_eris, t1=t1, t2=t2)[1:]
        elcorr_cc = ulnoccsd.get_fragment_energy(imp_eris, t1, t2, prjlo)
        if ccsd_t:
            from pyscf.lno.ulnoccsd_t_slow import kernel as UCCSD_T
            elcorr_cc_t = UCCSD_T(mcc, imp_eris, prjlo, t1=t1, t2=t2)
        else:
            elcorr_cc_t = 0.

    imp_eris = None

    return (elcorr_pt2, elcorr_cc, elcorr_cc_t), t1, t2

In [5]:
from collections.abc import Iterable
lno_type = ['1h'] * 2
mlno = ulnoccsd.ULNOCCSD_T(mf, lo_coeff, frag_lolist, frozen=nfrozen).set(verbose=4)
thresh = 1e-4
mlno.lno_thresh = [thresh*10,thresh]
lno_thresh = mlno.lno_thresh
lno_type = ['1h','1h'] if lno_type is None else lno_type
lno_thresh = [1e-5, 1e-6] if lno_thresh is None else lno_thresh
lno_pct_occ = None
lno_norb = None
lo_proj_thresh = 1e-10
lo_proj_thresh_active = 0.1
eris = None

nfrag = len(frag_lolist)
if lno_pct_occ is None:
    lno_pct_occ = [None, None]
if lno_norb is None:
    lno_norb = [[None,None]] * nfrag
mf = mlno._scf

if eris is None: eris = mlno.ao2mo()

# Loop over fragment
frag_res = [None] * nfrag
for ifrag,loidx in enumerate(frag_lolist):
    if ifrag == 0:
        if len(loidx) == 2 and isinstance(loidx[0], Iterable): # Unrestricted
            orbloc = [lo_coeff[0][:,loidx[0]], lo_coeff[1][:,loidx[1]]]
            lno_param = [
                [
                    {
                        'thresh': (
                            lno_thresh[i][s] if isinstance(lno_thresh[i], Iterable)
                            else lno_thresh[i]
                        ),
                        'pct_occ': (
                            lno_pct_occ[i][s] if isinstance(lno_pct_occ[i], Iterable)
                            else lno_pct_occ[i]
                        ),
                        'norb': (
                            lno_norb[ifrag][i][s] if isinstance(lno_norb[ifrag][i], Iterable)
                            else lno_norb[ifrag][i]
                        ),
                    } for i in [0, 1]
                ] for s in range(2)
            ]

        else:
            orbloc = lo_coeff[:,loidx]
            lno_param = [{'thresh': lno_thresh[i], 'pct_occ': lno_pct_occ[i],
                            'norb': lno_norb[ifrag][i]} for i in [0,1]]

        lno_coeff, frozen, uocc_loc, frag_msg = mlno.make_las(eris, orbloc, lno_type, lno_param)
        # frag_res[ifrag], frag_msg = mlno.impurity_solve(mf, mo_coeff, uocc_loc, eris, frozen=frozen)

        mo_occ = mlno.mo_occ
        frozen, maskact = ulnoccsd.get_maskact(frozen, [mo_occ[0].size, mo_occ[1].size])
        mcc = ulnoccsd.UCCSD(mf, mo_coeff=lno_coeff, frozen=frozen).set(verbose=mlno.verbose_imp)
        mcc._s1e = mlno._s1e
        mcc._h1e = mlno._h1e
        mcc._vhf = mlno._vhf
        if mlno.kwargs_imp is not None:
            mcc = mcc.set(**mlno.kwargs_imp)

        frag_res[ifrag], t1, t2 =\
            ulno_ccsd(mcc, lno_coeff, uocc_loc, mo_occ, maskact, ccsd_t=True)
    else: break

LO occ proj: 1 active | 0 standby | 4 orthogonal
LO occ proj: 0 active | 0 standby | 4 orthogonal

WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations



In [6]:
lno_result = np.array(frag_res[0])
print(lno_result)

[-5.43348892e-03 -9.11192861e-03 -2.65555052e-05]


In [7]:
emf = mf.e_tot
lno_dm = mf.make_rdm1(lno_coeff,mf.mo_occ)
mf.max_cycle = -1
mf.kernel(dm0=lno_dm)
print(emf)
print(mf.e_tot)



******** <class 'pyscf.df.df_jk.DFUHF'> ********
method = DFUHF
initial guess = minao
damping factor = 0
level_shift factor = 0
DIIS = <class 'pyscf.scf.diis.CDIIS'>
diis_start_cycle = 1
diis_space = 8
diis_damp = 0
SCF conv_tol = 1e-09
SCF conv_tol_grad = None
SCF max_cycles = -1
direct_scf = False
chkfile to save SCF result = /tmp/tmp0wk193ea
max_memory 16000 MB (current use 191 MB)
number electrons alpha = 5  beta = 4
Set gradient conv threshold to 3.16228e-05
init E= -4.73076167121335
  alpha nocc = 5  HOMO = -0.291859502230236  LUMO = 0.293201705097364
  beta  nocc = 4  HOMO = -0.360676881861952  LUMO = 0.169989731981045
converged SCF energy = -4.73076167121335  <S^2> = 1.0776962  2S+1 = 2.304514
-4.730761671213353
-4.730761671213351


In [8]:
# frozen = [[],[]]
nocc_a = int(sum(mf.mo_occ[0]))
actfrag_a = np.array([i for i in range(mol.nao) if i not in frozen[0]])
frzocc_a = np.array([i for i in range(nocc_a) if i in frozen[0]])
actocc_a = np.array([i for i in range(nocc_a) if i in actfrag_a])
actvir_a = np.array([i for i in range(nocc_a,mol.nao) if i in actfrag_a])
nfrzocc_a = len(frzocc_a)
nactocc_a = len(actocc_a)
nactvir_a = len(actvir_a)
nactorb_a = len(actfrag_a)
nocc_b = int(sum(mf.mo_occ[1]))
actfrag_b = np.array([i for i in range(mol.nao) if i not in frozen[1]])
frzocc_b = np.array([i for i in range(nocc_b) if i in frozen[1]])
actocc_b = np.array([i for i in range(nocc_b) if i in actfrag_b])
actvir_b = np.array([i for i in range(nocc_b,mol.nao) if i in actfrag_b])
nfrzocc_b = len(frzocc_b)
nactocc_b = len(actocc_b)
nactvir_b = len(actvir_b)
nactorb_b = len(actfrag_b)
print(nfrzocc_a,nactocc_a,nactvir_a,nactorb_a)
print(nfrzocc_b,nactocc_b,nactvir_b,nactorb_b)

3 2 4 6
2 2 3 5


In [9]:
from ad_afqmc.lno_afqmc import ulno_afqmc
options = {'n_eql': 5,
           'n_prop_steps': 50,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 98,
            'walker_type': 'uhf',
            'trial': 'uhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }
prjlo = [uocc_loc[0].T.conj(), uocc_loc[1].T.conj()]
ulno_afqmc.prep_afqmc(mf,lno_coeff,t1,t2,frozen,prjlo,
                   options,chol_cut=1e-5,
                   option_file='options.bin',
                   mo_file="mo_coeff.npz",
                   amp_file="amplitudes.npz",
                   chol_file="FCIDUMP_chol")

# Calculating Effective Active Space One-electron Integrals
# Generating Cholesky Integrals
# Composing AO ERIs from DF basis
# Finished calculating Cholesky integrals
# Size of the correlation space
# Number of electrons: (2, 2)
# Number of basis functions: (6, 5)
# Alpha Basis Cholesky shape: (81, 6, 6)
# Beta Basis Cholesky shape:  (81, 5, 5)


In [10]:
from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

ham_data, prop, trial, wave_data, sampler, options, _ = (
    ulno_afqmc._prep_afqmc())

# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Hostname: sharmagroup-rn
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
# Number of MPI ranks: 1
#
# norb: (6, 5)
# nelec: (2, 2)
#
# n_eql: 5
# n_prop_steps: 50
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_walkers: 1
# seed: 98
# walker_type: uhf
# trial: uhf
# dt: 0.005
# free_projection: False
# use_gpu: False
# n_exp_terms: 6
# orbital_rotation: True
# do_sr: True
# symmetry: False
# save_walkers: False
# ene0: 0.0
# n_batch: 1
#


In [11]:
import time
from jax import numpy as jnp
from jax import random
init_time = time.time()
comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

### initialize propagation
seed = options["seed"]
init_walkers = None
# dm_up = jnp.array(wave_data["mo_coeff"][0] @ wave_data["mo_coeff"][0].T.conj())
# dm_dn = jnp.array(wave_data["mo_coeff"][1] @ wave_data["mo_coeff"][1].T.conj())
# trial_rdm1 = [dm_up, dm_dn]
trial_rdm1 = trial.get_rdm1(wave_data)
if "rdm1" not in wave_data:
    wave_data["rdm1"] = trial_rdm1
ham_data = trial._build_measurement_intermediates(ham_data, wave_data)
ham_data = prop._build_propagation_intermediates(ham_data, trial, wave_data)

prop_data = prop.init_prop_data(trial, wave_data, ham_data, init_walkers)
if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
    raise ValueError(
        "Initial overlaps are zero. Pass walkers with non-zero overlap."
    )
prop_data["key"] = random.PRNGKey(seed + rank)

prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
prop_data["n_killed_walkers"] = 0
prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
print(prop_data["e_estimate"])

-4.730761671213349


In [ ]:
import opt_einsum as oe
from jax import vmap
def _calc_ecorr(trial, walker_up, walker_dn, ham_data, wave_data):
    '''uhf correlation energy'''
    # <HF|H-E0|walker>/<HF|walker>
    nocca, noccb = trial.nelec
    # rot_h1a, rot_h1b = ham_data['rot_h1']
    rot_chola, rot_cholb = ham_data['rot_chol']
    GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
    lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
                    GFa.T[nocca:,:nocca], backend="jax")
    lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
                    GFb.T[noccb:,:noccb], backend="jax")
    tr_lga = oe.contract('gii->g',lga)
    tr_lgb = oe.contract('gii->g',lgb)
    lglg_aa = oe.contract('g,g->',tr_lga,tr_lga) - oe.contract('gij,gji->',lga,lga)
    lglg_ab = oe.contract('g,g->',tr_lga,tr_lgb)*2
    lglg_bb = oe.contract('g,g->',tr_lgb,tr_lgb) - oe.contract('gij,gji->',lgb,lgb)
    ecorr = 0.5*(lglg_aa + lglg_ab + lglg_bb)
    return jnp.real(ecorr)

def _calc_eorb(trial, walker_up, walker_dn, ham_data, wave_data):
    '''uhf correlation energy'''
    # <HF|H-E0|walker>/<HF|walker>
    nocca, noccb = trial.nelec
    # rot_h1a, rot_h1b = ham_data['rot_h1']
    prjloa, prjlob = wave_data['prjlo']
    rot_chola, rot_cholb = ham_data['rot_chol']
    GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
    lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
                    GFa.T[nocca:,:nocca], backend="jax")
    lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
                    GFb.T[noccb:,:noccb], backend="jax")
    tr_lga = oe.contract('gii->g',lga)
    tr_lgb = oe.contract('gii->g',lgb)
    lga_orb = oe.contract('gik,ik->g',lga,prjloa)
    lgb_orb = oe.contract('gik,ik->g',lgb,prjlob)
    lglg_aa = oe.contract('g,g->',lga_orb,tr_lga) - oe.contract('gij,gjk,ik->',lga,lga,prjloa)
    lglg_ab = oe.contract('g,g->',lga_orb,tr_lgb) + oe.contract('g,g->',tr_lga,lgb_orb)
    lglg_bb = oe.contract('g,g->',lgb_orb,tr_lgb) - oe.contract('gij,gjk,ik->',lgb,lgb,prjlob)
    eorb = 0.5*(lglg_aa + lglg_ab + lglg_bb)
    return jnp.real(eorb)

In [ ]:
import jax
from jax import jvp, lax
def _t_olp(trial, walker_up, walker_dn, wave_data: dict) -> complex:
    '''
    <HF|t1+t2|walker>_i 
    = (C_ia <HF|i+ a|walker>/<HF|walker> + C_iajb <HF|i+ j+ a b|walker>/<HF|walker>) * <HF|walker>
    = (C_ia G_ia + C_iajb (G_ia G_jb-G_ib G_ja)) * <HF|walker>
    prj onto spin-orbit i
    '''
    # pa, pb = wave_data["prjlo"]
    nocca, noccb = trial.nelec
    t1a, t1b = wave_data["t1a"], wave_data["t1b"]
    t2aa, t2ab, t2bb = wave_data["t2aa"], wave_data["t2ab"], wave_data["t2bb"]
    greena, greenb = trial._calc_green(walker_up, walker_dn, wave_data)
    greena, greenb = greena[:nocca, nocca:], greenb[:noccb, noccb:]
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)
    o1 = oe.contract("ia,ia->", t1a, greena, backend="jax") \
            + oe.contract("ia,ia->", t1b, greenb, backend="jax")
    o2 = (oe.contract("iajb,ia,jb->", t2aa, greena, greena, backend="jax")
          + oe.contract("iajb,ia,jb->", t2bb, greenb, greenb, backend="jax")) * 0.5 \
                + oe.contract("iajb,ia,jb->", t2ab, greena, greenb, backend="jax")
    return (o1 + o2) * o0

def _t_orb(trial, walker_up, walker_dn, wave_data: dict) -> complex:
    '''
    <HF|t1+t2|walker>_i 
    = (C_ia <HF|i+ a|walker>/<HF|walker> + C_iajb <HF|i+ j+ a b|walker>/<HF|walker>) * <HF|walker>
    = (C_ia G_ia + C_iajb (G_ia G_jb-G_ib G_ja)) * <HF|walker>
    prj onto spin-orbit i
    '''
    pa, pb = wave_data["prjlo"]
    nocca, noccb = trial.nelec
    t1a, t1b = wave_data["t1a"], wave_data["t1b"]
    t2aa, t2ab, t2bb = wave_data["t2aa"], wave_data["t2ab"], wave_data["t2bb"]
    greena, greenb = trial._calc_green(walker_up, walker_dn, wave_data)
    greena, greenb = greena[:nocca, nocca:], greenb[:noccb, noccb:]
    o0 = trial._calc_overlap(walker_up,walker_dn,wave_data)
    o1 = oe.contract("ia,ka,ik->", t1a, greena, pa, backend="jax") \
            + oe.contract("ia,ka,ik->", t1b, greenb, pa, backend="jax")
    o2 = (oe.contract("iajb,ka,jb,ik->", t2aa, greena, greena, pa, backend="jax")
            + oe.contract("iajb,ka,jb,ik->", t2bb, greenb, greenb, pb, backend="jax")
            + oe.contract("iajb,ka,jb,ik->", t2ab, greena, greenb, pa, backend="jax")
            + oe.contract("jbia,jb,ka,ik->", t2ab, greena, greenb, pb, backend="jax")) * 0.5
    return (o1 + o2) * o0

    
def t_exp1(trial, x, h1_mod, walker_up, walker_dn, wave_data):
        '''
        unrestricted t_ia <psi_i^a|exp(x*h1_mod)|walker>/<HF|walker>
        '''
        walker_up_1x = walker_up + x * h1_mod[0].dot(walker_up)
        walker_dn_1x = walker_dn + x * h1_mod[1].dot(walker_dn)
        olp = _t_olp(walker_up_1x, walker_dn_1x, wave_data)
        return olp


def t_exp2(trial, x, chol_i, walker_up, walker_dn, wave_data) -> complex:
        '''
        t_ia <psi_i^a|exp(x*h2_mod)|walker>/<HF|walker>
        '''
        walker_up_2x = (
            walker_up
            + x * chol_i[0].dot(walker_up)
            + x**2 / 2.0 * chol_i[0].dot(chol_i[0].dot(walker_up))
        )
        walker_dn_2x = (
            walker_dn
            + x * chol_i[1].dot(walker_dn)
            + x**2 / 2.0 * chol_i[1].dot(chol_i[1].dot(walker_dn))
        )
        olp = _t_olp(walker_up_2x,walker_dn_2x,wave_data)
        return olp

def d2_exp2_i(trial, chol_i,walker, wave_data):
        x = 0.0
        f = lambda a: t_exp2(a,chol_i,walker,wave_data)
        _, d2f = jax.jvp(lambda x: jax.jvp(f, [x], [1.0])[1], [x], [1.0])
        return d2f


def th_corr(self, walker: jax.Array, ham_data: dict, wave_data: dict):
        '''
        <HF|(t1+t2) (H-E0)|walker>/<HF|walker>
        '''

        norb = self.norb
        chol = ham_data["chol"].reshape(-1, norb, norb)
        h1_mod = ham_data['h1_mod']
        h0_E0 = ham_data["h0"]-ham_data["E0"]

        nocc = walker.shape[1]
        o0 = jnp.linalg.det(walker[: nocc, :]) ** 2

        x = 0.0
        # one body
        f1 = lambda a: t_exp1(a,h1_mod,walker,wave_data)
        olp_orb12, d_overlap = jvp(f1, [x], [1.0])

        # two body
        def scanned_fun(carry, c):
            walker, wave_data = carry
            return carry, d2_exp2_i(c,walker,wave_data)

        _, d2_olp2_i = lax.scan(scanned_fun, (walker, wave_data), chol)
        d_2_overlap = jnp.sum(d2_olp2_i)/2

        # <hf|(t1+t2)_i (h1+h2)|walker>/<hf|walker>
        e_orb12 = (h0_E0*olp_orb12 + d_overlap + d_2_overlap) / o0
        t_orb12 = olp_orb12 /o0 # <(t1+t2)_i>

        return jnp.real(e_orb12), jnp.real(t_orb12)

In [32]:
walker_up, walker_dn = prop_data['walkers'][0][0], prop_data['walkers'][1][0]

In [36]:
walker_up_r = np.random.randn(*walker_up.shape)
walker_dn_r = np.random.randn(*walker_dn.shape)
ecorr = _calc_ecorr(trial, walker_up_r, walker_dn_r, ham_data, wave_data)
eorb = _calc_eorb(trial, walker_up_r, walker_dn_r, ham_data, wave_data)
e_ref = trial._calc_energy(walker_up_r, walker_dn_r, ham_data, wave_data)
print(eorb)
print(ecorr)
print(jnp.real(e_ref) - emf)

0.4091269673929979
0.8149628578785431
0.8149616665566066


In [ ]:
nocca, noccb = trial.nelec
prjloa, prjlob = wave_data['prjlo']
rot_chola, rot_cholb = ham_data['rot_chol']
GFa, GFb = trial._calc_green(walker_up, walker_dn, wave_data)
lga = oe.contract('gij,jk->gik', rot_chola[:,:nocca,nocca:],
                GFa.T[nocca:,:nocca], backend="jax")
lgb = oe.contract('gij,jk->gik', rot_cholb[:,:noccb,noccb:],
                GFb.T[noccb:,:noccb], backend="jax")
tr_lga = oe.contract('gii->g',lga)
tr_lgb = oe.contract('gii->g',lgb)
lga_orb = oe.contract('gik,ik->g',lga,prjloa)
lgb_orb = oe.contract('gik,ik->g',lgb,prjlob)
lglg_aa = oe.contract('g,g->',lga_orb,tr_lga) - oe.contract('gij,gjk,ik->',lga,lga,prjloa)
lglg_ab = oe.contract('g,g->',lga_orb,tr_lgb)
lglg_ba = oe.contract('g,g->',tr_lga,lgb_orb)
lglg_bb = oe.contract('g,g->',lgb_orb,tr_lgb) - oe.contract('gij,gjk,ik->',lgb,lgb,prjlob)
eorb = 0.5*(lglg_aa + lglg_ab + lglg_bb)

In [39]:
print(prjloa)
print(prjlob)

[[ 1.00000000e+00 -5.10962800e-15]
 [-5.10962800e-15  2.61082983e-29]]
[[0. 0.]
 [0. 0.]]


In [55]:
t1a, t1b = t1
t2aa, t2ab, t2bb = t2
t2aa = (t2aa - t2aa.transpose(0, 1, 3, 2)) / 2
t2bb = (t2bb - t2bb.transpose(0, 1, 3, 2)) / 2
t2aa = t2aa.transpose(0, 2, 1, 3)
t2ab = t2ab.transpose(0, 2, 1, 3)
t2bb = t2bb.transpose(0, 2, 1, 3)
print(t1a.shape, t1b.shape)
print(t2aa.shape)
print(t2ab.shape)
print(t2bb.shape)
wave_data['t1a'], wave_data['t1b'] = t1a, t1b
wave_data['t2aa'], wave_data['t2ab'], wave_data['t2bb'] \
    = t2aa, t2ab, t2bb

(2, 4) (2, 3)
(2, 4, 2, 4)
(2, 4, 2, 3)
(2, 3, 2, 3)


In [54]:
GFa, GFb = trial._calc_green(walker_up_r, walker_dn_r, wave_data)
GFa, GFb = GFa[:nocca, nocca:], GFb[:noccb, noccb:]
ot2aa = oe.contract('iajb,ia,jb->',t2aa,GFa,GFa)
ot2aa_ex = oe.contract('iajb,ib,ja->',t2aa,GFa,GFa)
ot2ab = oe.contract('iajb,ia,jb->',t2ab,GFa,GFb)
ot2ba = oe.contract('jbia,jb,ia->',t2ab,GFa,GFb)
ot2bb = oe.contract('iajb,ia,jb->',t2bb,GFb,GFb)
ot2bb_ex = oe.contract('iajb,ib,ja->',t2bb,GFb,GFb)
print(ot2aa,ot2aa_ex)
print(ot2ab,ot2ba)
print(ot2bb,ot2bb_ex)

-0.09681721373534696 0.09681721373534696
-0.24841543499427354 -0.24841543499427354
-0.19655792219408952 0.19655792219408952


In [58]:
t12 = _t_olp(trial, walker_up, walker_dn, wave_data)
print(t12)

(-0+0j)
